# Imports

In [1]:
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset, Dataset
from transformers import (AutoTokenizer,
                          AutoConfig,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments)
import transformers
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
import pandas as pd
from trl import SFTTrainer
from peft.tuners.lora import LoraLayer
import datetime


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/matheusalb/anaconda3/envs/llm/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


-----

# Funções

## Carregamento modelo

In [2]:
def create_and_prepare_model(model_name):
    # compute_dtype = getattr(torch, "float16")
    
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"": 0},
        trust_remote_code=True,
    )

    peft_config =  LoraConfig(
        lora_alpha=16, # 16
        lora_dropout=0.1, # 0.05
        bias="none",
        task_type="CAUSAL_LM",
        r=64, # 8
        target_modules=[
            "q_proj",
            "v_proj",
            # "k_proj",
            # "o_proj"
        ]
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.eos_token
    
    return model, peft_config, tokenizer


## Cria dataset

In [3]:
def generate_prompt(data_point):
    instruction = 'Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante'
    prompt = f'''\
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{data_point['comentario']}
### Response:
{data_point['sugestaoResposta']}
'''.strip()               
    return prompt

def get_dataset(path, final_string):
    messages = {}
    messages['text'] = [] 
              
    df = pd.read_csv(path, skip_blank_lines=True)
    df = df.dropna(how='all')
    for _, linha in df.iterrows():
        text = generate_prompt(linha)
        messages['text'].append(text)
    
    return Dataset.from_dict(messages)

-----

# Execução

In [4]:
model_name = 'openlm-research/open_llama_7b_v2'
train_path = '../../../data/base_2k/train_base.csv'
validation_path = '../../../data/base_2k/validation_base.csv'    
OUTPUT_DIR = './results/qlora_openLLaMA_SFT_base_2k_'+datetime.datetime.now().isoformat()

training_arguments = transformers.TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=2, # micro_batch_size
    gradient_accumulation_steps=4, # batch_size / micro_batch_size
    # num_train_epochs=80,
    max_steps=100,
    save_steps=100,
    optim="paged_adamw_32bit", # adamw_torch
    save_total_limit=3,
    logging_steps=10,
    learning_rate=2e-4, # 3e-4
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03, #0.05
    # warmup_steps=100,
    lr_scheduler_type="constant", #cosine
    evaluation_strategy='steps',
    eval_steps=20,
    report_to="wandb"
)

model, peft_config, tokenizer = create_and_prepare_model(model_name)
# se não fizer isso vai disparar warning, ativar para inferência 
model.config.use_cache = False

train_data = get_dataset(train_path, tokenizer.eos_token)
validation_data = get_dataset(validation_path, tokenizer.eos_token)

# Supervised finetunning é popularmente conhecido como instruction finetuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=250,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)
# data_collator=transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
# testar com data collector


Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.73s/it]
/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [5]:
# import os
# os.environ['WANDB_NOTEBOOK_NAME'] = 'TESTE_2'
trainer.train()
print('Path:', OUTPUT_DIR)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: matheusalb. Use `wandb login --relogin` to force relogin


 10%|█         | 10/100 [00:40<05:52,  3.91s/it]

{'loss': 2.1695, 'learning_rate': 0.0002, 'epoch': 0.09}


 20%|██        | 20/100 [01:20<05:09,  3.87s/it]

...

{'loss': 1.0988, 'learning_rate': 0.0002, 'epoch': 0.89}


                                                 
100%|██████████| 100/100 [07:41<00:00,  3.91s/it]

{'eval_loss': 1.0708171129226685, 'eval_runtime': 12.3663, 'eval_samples_per_second': 8.086, 'eval_steps_per_second': 1.051, 'epoch': 0.89}


100%|██████████| 100/100 [07:47<00:00,  4.68s/it]

{'train_runtime': 472.7698, 'train_samples_per_second': 1.692, 'train_steps_per_second': 0.212, 'train_loss': 1.2912306213378906, 'epoch': 0.89}
Path: ./results/qlora_openLLaMA_SFT_base_2k_2023-07-19T23:34:26.150155


In [6]:
def gen_input(comment):
    prompt = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
{comment}
### Response:
'''           
    return prompt

In [7]:
generation_config=model.generation_config
generation_config.max_new_tokens=300
# generation_config.temeperature=0.7
# generation_config.top_k=0.9
generation_config.num_return_sequences=1
# generation_config.pad_token_id=tokenizer.eos_token_id
# generation_config.eos_token_id=tokenizer.eos_token_id

def inference(model, text):
    enconded = tokenizer(text, return_tensors="pt")
    outputs = model.generate(
        input_ids=enconded.input_ids,
        attention_mask=enconded.attention_mask,
        generation_config=generation_config
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [8]:
comment = 'Alimentação caríssima para um péssimo atendimento.. super mal atendido'
inference(model, gen_input(comment))

/home/matheusalb/anaconda3/envs/llm/lib/python3.11/site-packages/transformers/generation/utils.py:1448: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Alimentação caríssima para um péssimo atendimento.. super mal atendido
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos que a sua comida não tenha atendido às suas expectativas em relação ao atendimento. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor. Agradecemos por compartilhar sua opinião.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Alimentação caríssima para um péssimo atendimento.. super mal 

In [9]:
#treino
comment='Filé estava duro. Ficamos decepcionados.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Filé estava duro. Ficamos decepcionados.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que a comida não tenha atendido às suas expectativas. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por compartilhar sua experiência conosco e esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor. Agradecemos por compartilhar sua opinião.
### Comentário de um cliente ao seu restaurante
### Input:
O atendimento é muito ruim. Não temos o mesmo nome do cliente.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que a experiência não tenha atendido às suas expectativas. Vamos revisar nossos processos para garantir q

In [10]:
comment='''A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
A falta de vaga para cadeirantes e idosos é uma falha terrível nos dias de hoje, além de ser lei não tivemos nenhum apoio ou mesmo interesse por parte dos funcionários para ajudar na questão do cadeirante que estava nos acompanhado.
Não vou deixar de avaliar os outros pontos, no que tange ao atendimento interno e as delícias que são servidas. tudo de bom. 
Mas não vá com cadeirante que passará sufoco.
### Response:
Olá, agradecemos pelo seu feedback e lamentamos pela experiência negativa com a falta de vaga para cadeirantes e idosos. Entendemos que isso pode causar muita dor e preocupação para os clientes. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto ao atendim

In [11]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Response:
Olá,

Lamentamos muito pela má experiência que você teve em nosso restaurante. Pedimos desculpas pelo atendimento demorado e pela falta de gosto no prato de frango. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.

Atenciosamente,

Equipe do Restaurante 100% Brasileiro

### Comentário:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Resposta:
Olá,

Lamentam

In [12]:
comment = 'Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Response:
Olá,

Lamentamos muito pela má experiência que você teve em nosso restaurante. Pedimos desculpas pelo atendimento demorado e pela falta de gosto no prato de frango. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-la novamente para oferecer uma experiência melhor.

Atenciosamente,

Equipe do Restaurante 100% Brasileiro

### Comentário:
Horrível! Restaurante vazio e o atendimento ainda demora. Pedi um frango e veio com gosto de de peixe. Nojo!
### Resposta:
Olá,

Lamentam

In [13]:
comment = 'Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Lugar estava quente e, apesar de poucos usuários, a comida demorou bastante a ficar pronta.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que a comida tenha demorado a ficar pronta. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto ao ambiente, estamos trabalhando para melhorar a qualidade do serviço e a comodidade dos nossos clientes. Agradecemos por compartilhar sua experiência e esperamos ter a oportunidade de recebê-la novamente.
### Comentário de um cliente ao seu restaurante
### Input:
O lugar é muito bonito, mas a comida é muito cara e não tem muito escolha.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que a comida tenha sido ca

In [14]:
comment = 'Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Com o aumento do público nos finais de semana, poderiam conciliar o atendimento na área interna com a área externa, pois quem fica do lado de fora espera cerca de 40 min pra ser atendido.
### Response:
Olá, agradecemos pelo seu feedback. Lamentamos que tenha experimentado uma demora no atendimento na área externa. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por nos informar e esperamos ter a oportunidade de recebê-lo novamente em nosso restaurante. Agradecemos por compartilhar sua experiência conosco.

Agradecemos por compartilhar sua opinião.

Agradecemos pelo seu feedback. Lamentamos que tenha experimentado uma demora no atendimento na área interna. Vam

In [15]:
comment = 'O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!'
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
O pior yakisoba que eu já comi na vida! Vem numa caixa com a massa toda amassada e o molho é horrível! Tive que jogar fora! Parecia comida de cachorro! Nunca mais!
### Response:
Olá, agradecemos pelo seu feedback sobre a comida do nosso restaurante. Lamentamos muito que a sua experiência não tenha sido satisfatória. Vamos revisar nossos processos para garantir que a massa e o molho sejam preparados de forma mais adequada. Agradecemos por nos informar sobre a amassagem da massa e esperamos que isso não se repita. Agradecemos por nos informar sobre o molho e vamos revisar nossos processos para melhorar a qualidade do mesmo. Agradecemos por nos informar e esperamos que a próxima vez possamos oferecer um

In [16]:
comment= '''Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Cheguei para tomar café as 7h da manhã, pouquíssimas pessoas ,pedi um omelete levou 23 minutos a essa altura meu café já havia acabado, e chegou uma pessoa depois de mim e o pedido dela que era muito mais que o meu simples omelete saiu primeiro.
Péssimo atendimento .
Já fui bem atendida por outros  funcionários  mais essa turma da manhã 3 pessoas nota 2.
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência no nosso restaurante. Lamentamos muito pela má atendimento que você recebeu durante a sua visita. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-la novamente para ofe

In [17]:
#treino
comment='Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!'

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Não recomendo! Não faz muito meu estilo cantar para quem não conheço, coisa que é impossível sem marcar antes. Sim, existem vários ambientes tocando estilos diferentes mas é necessário muita sorte para conseguir um lugar onde todos gostem do que está tocando. Sem falar que, se tratando do karaokê: ninguém é profissional por isso as performances são horríveis e a musica e letra são retiradas do YouTube, mas mesmo assim o espaço insiste em cobrar entrada! Para finalizar a comida, que é bem cara, veio fria e não gostei de nada que comi! Nem a bebida salvou minha noite, paguei um absurdo pela menor porção de caipirinha que já vi! Teria feito melhor na minha casa!
### Response:
Olá, agradecemos pelo seu f

In [18]:
#treino
comment = '''Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.'''
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Eu gosto muito de tomar café aos domingos nessa panificadora, porém não sei o que esta acontecendo, ela está ficando muito suja.
Gosto da parte de cima,mas quando chego as mesas estão todas sujas, com xícaras, resto de lanches e etc.
Acredito que não estão higienizando.
Outro detalhe a tapioca está vindo muito salgada.
Espero que melhore, para que possamos continuar frequentando.
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos que a sua visita não tenha sido todavia satisfatória. Agradecemos por nos informar sobre a sujaza das mesas e a tapioca. Vamos analisar esses pontos para melhorar nossos serviços. Agradecemos por compartilhar sua opinião

In [19]:
comment = '''Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer seu carater?
Tudo bem que a mão de obra de estrangeiros seja mais barata. Mas vocês teriam que pensar no bem estar dos clientes.
Descobri o nome do garçom.
CARLOS AMED. Ele é cubano.
Ia fazer uma denuncia por assedio mas pensei na empresa. Por favor selecione melhor seus colaboradores.
Muito indignada.'''

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Eu gostaria de deixar minha indignação quanto o atendimento de um garçom.
Não estou  reclamando do restaurante.
Estive no restaurante umas duas vezes, foi tudo muito bom.
Mas me decepcionei na minha ultima vez.
Foi atendida por um garçom mal educado e sem ética pois o mesmo alem de ser Grosso ficava olhando para as minhas pernas. Fiquei muito constrangida com a situação.
Almoçamos tomamos. Sucos e cervejas, mesmo porque estavamos em 6 pessoas.
No final pedimos a conta e o mesmo  trouxe duas contas uma com mais itens incluindo a refeiçao, outra somente com as bebidas,achei estranho porque tinham me cobrado normal das últimas vezes. Em fim.
Como um gestor de um restaurante contrata pessoas sem conhecer

In [20]:
comment = "Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar."
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Fui no dia 10/12/21 às 11:00 o restaurante não tinha ninguém ainda, fui fazer um pedido para viagem. Depois de quase 30 minutos de espera percebo que tinham clientes que chegaram depois comendo...questionei um garçom pela demora do pedido foi averiguar e não deu nenhuma satisfação. Aí percebi depois que o pedido estava no balcão aguardando o favor de algum garçom pegar e olha que nem estava lotado o restaurante...desrespeito com o cliente e garçons mal treinados.. nunca mais nesse lugar.
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência no nosso restaurante. Lamentamos pela demora no entregamento do seu pedido e pela falta de atendimento ao cliente. Vamos revisar nossos process

In [21]:
comment = "Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!"
inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Só não dou zero porque não dá e porque a torta de frango estava boazinha. Priorizam quem tem aparência de ter mais dinheiro e os garçons não fazem questão alguma de atender as pessoas. Olharam na minha cara, passaram direto até que eu implorasse pra ser atendida. :) Não anotaram meu pedido e ainda queriam colocar a culpa em mim pelo esquecimento. Enfim, não volto!
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos que a torta de frango tenha sido boazinha. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto ao atendimento, vamos reforçar a importância de atendimento às clientes. Quanto ao pedido, vamos revisar nos

In [22]:
comment = "Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo"

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Pedimos um prato de bife ao molho madeira sendo que o feijão veio com gosto de molho madeira e o bife veio com gosto de queijo então eu recomendaria que vocês se dedicassem mais e as atende vistas são muito de humor baixo
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência no nosso restaurante. Lamentamos que a comida não tenha atendido às suas expectativas em relação ao molho madeira e ao gosto do feijão. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Quanto ao humor das vistas, iremos trabalhar para melhorar essa área. Agradecemos por compartilhar sua opinião e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante. Agradecemos p

In [23]:
comment = "Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo."

inference(model, gen_input(comment))

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
Escreva, em Português, um comentário de resposta ao seguinte comentário de um cliente ao seu restaurante
### Input:
Pedi no ifood camarão empanado e mandaram camarão alho e óleo muito sem graça e poucas unidades. Fui inventar de experimentar por causa da propaganda no Instagram e me ferrei! Não recomendo.
### Response:
Olá, agradecemos pelo seu feedback sobre a sua experiência em nosso restaurante. Lamentamos muito pela má impressão causada pelo camarão empanado que você pediu. Vamos revisar nossos processos para garantir que isso não aconteça novamente. Agradecemos por nos informar sobre a falta de graça e de quantidade. Entendemos sua preocupação e vamos trabalhar para melhorar nossos serviços. Agradecemos por compartilhar sua experiência e esperamos ter a oportunidade de recebê-la novamente em nosso restaurante